In [ ]:
from vw_executor.vw import Vw
from vw_executor.vw_opts import Grid
from vw_sim.simulation import Simulation
from vw_sim.example import Regression
from vw_sim.core import ExampleFactory
from vw_sim.utils import save
import random

vw = Vw('.cache', 'vw')

In [ ]:
sim = Simulation.run(
    ExampleFactory(
        features = {
            'a': {
                'x': lambda _: random.random() * 10,
                'y': lambda _: random.random()
            }
        },
        label = lambda obj, _: obj['a']['x']*obj['a']['y'] + 2 * obj['a']['x'] + 1,
        example_type = Regression
    ),
    10000
)
path = save(sim.txt, folder='data')
path

In [ ]:
import pandas as pd
results = vw.train(
    path,
    pd.DataFrame(Grid({
        '#args': ['--coin', '--freegrad', '--learning_rate 0.1', '--learning_rate 0.5', '--learning_rate 2'],
        '#interactions': ['', '-q aa']
    })))
results